In [1]:
!pip install -q pyomo
from pyomo.environ import *
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.5 MB/s eta 0:00:00


In [2]:
# total drone types
n = 5
#capacity array
t = np.array([12,14,10,15,14])
#cost array
c = np.array([6,7.5,9.5,10,12.5])
#parking array
p = np.array([4.25, 3.25, 2.5, 3.45,2.25])

In [3]:
model = ConcreteModel()
model.x = Var(np.arange(n), domain = NonNegativeIntegers)
model.obj = Objective(expr= sum(model.x[i]*t[i] for i in range(n)), sense = maximize)
model.constraints = ConstraintList()
model.constraints.add(expr = sum(model.x[i]*p[i] for i in range(n)) <= 73)
model.constraints.add(expr = sum(model.x[i]*c[i] for i in range(n)) <= 59)
model.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers
          4 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 14*x[4

In [4]:
# install cbc solver
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')

# solving your pipeModel using cbc solver
results = optCBC.solve(model)

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [5]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 114.0
  Upper bound: 114.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 5
  Number of binary variables: 0
  Number of integer variables: 5
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.03
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.15739846229553223
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [6]:
model.obj()

114.0

In [7]:
for i in range(n):
  print("x",i,": ",model.x[i].value)

x 0 :  6.0
x 1 :  3.0
x 2 :  0.0
x 3 :  0.0
x 4 :  0.0


In [8]:
#Relax the above-constructed formulation to a linear program and solve for optimality. Report the
#solution for the relaxed program and compare it with the solution MILP formulation.

In [9]:
model1 = ConcreteModel()
model1.x = Var(np.arange(n), domain = NonNegativeReals)
model1.obj = Objective(expr= sum(model1.x[i]*t[i] for i in range(n)), sense = maximize)
model1.constraints = ConstraintList()
model1.constraints.add(expr = sum(model1.x[i]*p[i] for i in range(n)) <= 73)
model1.constraints.add(expr = sum(model1.x[i]*c[i] for i in range(n)) <= 59)
model1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 14*x[4]

1 Constraint

In [10]:
# solving your pipeModel using cbc solver
results1 = optCBC.solve(model1)
print(results1)


Problem: 
- Name: unknown
  Lower bound: 118.0
  Upper bound: 118.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 5
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 3
  Error rc: 0
  Time: 0.060895681381225586
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [11]:
print(model1.obj())
for i in range(n):
  print("x",i,": ",model1.x[i].value)

117.9999996
x 0 :  9.8333333
x 1 :  0.0
x 2 :  0.0
x 3 :  0.0
x 4 :  0.0


Changing Righthand side constraints

In [12]:
model2 = ConcreteModel()
model2.x = Var(np.arange(n), domain = NonNegativeIntegers)
model2.obj = Objective(expr= sum(model2.x[i]*t[i] for i in range(n)), sense = maximize)
model2.constraints = ConstraintList()
model2.constraints.add(expr = sum(model2.x[i]*p[i] for i in range(n)) <= 87)
model2.constraints.add(expr = sum(model2.x[i]*c[i] for i in range(n)) <= 71)
model2.pprint()
# solving your pipeModel using cbc solver
results2 = optCBC.solve(model2)
print(results2)
print(model2.obj())
for i in range(n):
  print("x",i,": ",model2.x[i].value)

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers
          4 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 14*x[4

In [13]:
model3 = ConcreteModel()
model3.x = Var(np.arange(n), domain = NonNegativeReals)
model3.obj = Objective(expr= sum(model3.x[i]*t[i] for i in range(n)), sense = maximize)
model3.constraints = ConstraintList()
model3.constraints.add(expr = sum(model3.x[i]*p[i] for i in range(n)) <= 87)
model3.constraints.add(expr = sum(model3.x[i]*c[i] for i in range(n)) <= 71)
model3.pprint()
# solving your pipeModel using cbc solver
results3 = optCBC.solve(model3)
print(results3)
print(model3.obj())
for i in range(n):
  print("x",i,": ",model3.x[i].value)

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 14*x[4]

1 Constraint

Changing simultaneously by 4 values


In [14]:
#MILP
k = np.array([69, 67, 65, 63])
l = np.array([88, 89, 90, 91])
optimal = []
values = []
n=5
cmc = [None]*4


In [15]:
for j in range(4):
  cmc[j] = ConcreteModel()
  cmc[j].x = Var(np.arange(n), domain = NonNegativeIntegers)
  cmc[j].obj = Objective(expr= sum(cmc[j].x[i]*t[i] for i in range(n)), sense = maximize)
  cmc[j].constraints = ConstraintList()
  cmc[j].constraints.add(expr = sum(cmc[j].x[i]*p[i] for i in range(n)) <= l[j])
  cmc[j].constraints.add(expr = sum(cmc[j].x[i]*c[i] for i in range(n)) <= k[j])
  # cmc[j].pprint()
  # solving your pipeModel using cbc solver
  results = optCBC.solve(cmc[j])
  optimal.append(cmc[j].obj())
  values.append([cmc[j].x[i].value for i in range(n)])

In [16]:
optimal

[136.0, 132.0, 126.0, 124.0]

In [17]:
values

[[9.0, 2.0, 0.0, 0.0, 0.0],
 [11.0, 0.0, 0.0, 0.0, 0.0],
 [7.0, 3.0, 0.0, 0.0, 0.0],
 [8.0, 2.0, 0.0, 0.0, 0.0]]

In [18]:
## Relaxing LP

optimalr = []
valuesr= []
n=5
cmcr = [None]*4

for j in range(4):
  cmcr[j] = ConcreteModel()
  cmcr[j].x = Var(np.arange(n), domain = NonNegativeReals)
  cmcr[j].obj = Objective(expr= sum(cmcr[j].x[i]*t[i] for i in range(n)), sense = maximize)
  cmcr[j].constraints = ConstraintList()
  cmcr[j].constraints.add(expr = sum(cmcr[j].x[i]*p[i] for i in range(n)) <= l[j])
  cmcr[j].constraints.add(expr = sum(cmcr[j].x[i]*c[i] for i in range(n)) <= k[j])
  # cmcr[j].pprint()
  # solving your pipeModel using cbc solver
  results = optCBC.solve(cmcr[j])
  optimalr.append(cmcr[j].obj())
  valuesr.append([cmcr[j].x[i].value for i in range(n)])

In [19]:
optimalr

[138.0, 134.000004, 129.999996, 126.0]

In [20]:
valuesr

[[11.5, 0.0, 0.0, 0.0, 0.0],
 [11.166667, 0.0, 0.0, 0.0, 0.0],
 [10.833333, 0.0, 0.0, 0.0, 0.0],
 [10.5, 0.0, 0.0, 0.0, 0.0]]